In [1]:
import requests
import pandas as pd
import json
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from web3 import Web3
from ens import ENS
import pytz
import ast
from datetime import datetime

In [2]:
# Constants
URL = "https://hub.snapshot.org/graphql"
HEADERS = {"Content-Type": "application/json"}
CSV_INPUT_PATH = '../Data/Proposals_Data.csv'
CSV_OUTPUT_PATH = '../Data/Votes_Data.csv'
MAX_WORKERS = 5 
FIRST = 1000

# Load proposal IDs
proposals_data = pd.read_csv(CSV_INPUT_PATH, encoding='latin1')
proposal_ids = proposals_data['proposal_id'].unique()

# Output CSV header
CSV_HEADER = [
    "vote_id", 
    "voter_address", 
    "voting_power",
    "vote_timestamp",
    "proposal_id", 
    "proposal_title", 
    "proposal_type", 
    "proposal_choices", 
    "proposal_creation_date", 
    "proposal_start_date", 
    "proposal_end_date", 
    "choice", 
    "reason",
    "space"
]

# GraphQL Query Template
QUERY = """
query($lastCreated: Int, $proposalId: String){
  votes (
    first: 1000
    skip: 0
    where: {
      proposal: $proposalId
      created_gt: $lastCreated
    }
    orderBy: "created",
    orderDirection: asc
  ) {
    id
    voter
    vp
    created
    proposal {
      id
      title
      type
      choices
      created
      start
      end
      space {
        id
      }
    }
    choice
    reason
  }
}
"""

# Function to fetch all votes for a single proposal
def fetch_votes_for_proposal(proposal_id):
    all_votes = []
    last_created = 0

    while True:
        variables = {"lastCreated": last_created, "proposalId": proposal_id}
        response = requests.post(URL, json={"query": QUERY, "variables": variables}, headers=HEADERS)

        if response.status_code != 200:
            print(f"Failed to fetch for proposal {proposal_id}: {response.status_code}")
            break

        votes = response.json().get("data", {}).get("votes", [])
        if not votes:
            break

        all_votes.extend(votes)
        last_created = votes[-1]['created']

    return all_votes

# Main function to process proposals in parallel and write to CSV
def main():
    with open(CSV_OUTPUT_PATH, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(CSV_HEADER)

        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = {executor.submit(fetch_votes_for_proposal, pid): pid for pid in proposal_ids}

            for future in as_completed(futures):
                proposal_id = futures[future]
                try:
                    votes = future.result()
                    for vote in votes:
                        row = [
                            vote['id'],
                            vote['voter'],
                            vote['vp'],
                            vote['created'],
                            vote['proposal']['id'],
                            vote['proposal']['title'],
                            vote['proposal']['type'],
                            vote['proposal']['choices'],
                            vote['proposal']['created'],
                            vote['proposal']['start'],
                            vote['proposal']['end'],
                            vote['choice'],
                            vote['reason'],
                            vote['proposal']['space']['id']
                        ]
                        writer.writerow(row)
                    print(f"Fetched and saved votes for Proposal ID {proposal_id}")
                except Exception as e:
                    print(f"Error processing proposal {proposal_id}: {e}")

if __name__ == "__main__":
    main()


Fetched and saved votes for Proposal ID 0xda3c4a1503a2b90423a69339d4b3d5393a44ef2befd3eaa57aa0f24eda7883a9
Fetched and saved votes for Proposal ID 0xaed836747d669ec0a9459e1c15a2a40621e066a9230e32ce830b6047dd8c70b0
Fetched and saved votes for Proposal ID 0xfb33f167d65de61dae6a88c91beaf3cc0add78e7980118abf5e0ab3e364113fd
Fetched and saved votes for Proposal ID 0xd07e5c85dae5f59dc444838cbfba8ecf6bd5cabdc3e3787bfa7559877bfd2b1d
Fetched and saved votes for Proposal ID 0xcadd1fcfd66913a953a12a9d07d052d0ff97ec16932c407a5582c66f279d5808
Fetched and saved votes for Proposal ID 0x27f06b093b34f8a8b7a23175343b051c39de4ec4726389bfcc905c14d340d936
Fetched and saved votes for Proposal ID 0x446b08599cd0a3956589e82b5eaed8bb8bbfa570671cca436fafb1b316d0d06c
Fetched and saved votes for Proposal ID 0x52d4bc671fb09118ec727f411b5d0740ce539c06a2b742adf1bc90cbb67cc4f8
Fetched and saved votes for Proposal ID 0x9c80e85a7e5a69fd0a919c6a161a1db007103d55348a7ff62967be6df59c7611
Fetched and saved votes for Proposal 

In [3]:
votes = pd.read_csv("../Data/Votes_Data.csv")

In [4]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   vote_id                 1111 non-null   object
 1   voter_address           1111 non-null   object
 2   voting_power            1111 non-null   int64 
 3   vote_timestamp          1111 non-null   int64 
 4   proposal_id             1111 non-null   object
 5   proposal_title          1111 non-null   object
 6   proposal_type           1111 non-null   object
 7   proposal_choices        1111 non-null   object
 8   proposal_creation_date  1111 non-null   int64 
 9   proposal_start_date     1111 non-null   int64 
 10  proposal_end_date       1111 non-null   int64 
 11  choice                  1111 non-null   object
 12  reason                  182 non-null    object
 13  space                   1111 non-null   object
dtypes: int64(5), object(9)
memory usage: 121.6+ KB


In [5]:
# Define IST timezone
ist = pytz.timezone('Asia/Kolkata')

# Convert the timestamp columns to UTC, then convert to IST
for col in ['vote_timestamp', 'proposal_creation_date', 'proposal_start_date', 'proposal_end_date']:
    votes[col] = pd.to_datetime(votes[col], unit='s', utc=True).dt.tz_convert(ist)

# Format datetime columns as string with timezone offset
for col in ['vote_timestamp', 'proposal_creation_date', 'proposal_start_date', 'proposal_end_date']:
    votes[col] = votes[col].dt.strftime('%Y-%m-%d %H:%M:%S%z')


In [6]:
# Add source column
votes['source'] = votes['space'].map({
    'citizenshouse.eth': 'citizen_house_vote',
    'retrofunding.eth': 'retro_funding_vote'
}).fillna('other')

# Add document_role column
votes['document_role'] = 'vote_record'

In [7]:
# Convert 'proposal_choices' string to list
votes['proposal_choices'] = votes['proposal_choices'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Convert 'choice' string to a list of integers
votes['chosen_option_index'] = votes['choice'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else [int(x)]
)

# Map each index to its corresponding option text
votes['chosen_option_text'] = votes.apply(
    lambda row: [row['proposal_choices'][i - 1] for i in row['chosen_option_index']]
    if isinstance(row['proposal_choices'], list) else None,
    axis=1
)

In [8]:
votes.columns

Index(['vote_id', 'voter_address', 'voting_power', 'vote_timestamp',
       'proposal_id', 'proposal_title', 'proposal_type', 'proposal_choices',
       'proposal_creation_date', 'proposal_start_date', 'proposal_end_date',
       'choice', 'reason', 'space', 'source', 'document_role',
       'chosen_option_index', 'chosen_option_text'],
      dtype='object')

In [9]:
retro_votes = votes[votes['space'] == 'retrofunding.eth']

In [10]:
retro_votes[['proposal_choices', 'choice', 'chosen_option_index', 'chosen_option_text']]

,proposal_choices,choice,chosen_option_index,chosen_option_text
210,"[Arcturus, Bellatrix, Canopus]",[1],[1],[Arcturus]
211,"[Arcturus, Bellatrix, Canopus]",[1],[1],[Arcturus]
212,"[Arcturus, Bellatrix, Canopus]",[1],[1],[Arcturus]
213,"[Arcturus, Bellatrix, Canopus]",[1],[1],[Arcturus]
214,"[Arcturus, Bellatrix, Canopus]","[1, 3]","[1, 3]","[Arcturus, Canopus]"
...,...,...,...,...
539,"[Superscale, Acceleratooor, Goldilocks]",[3],[3],[Goldilocks]
540,"[Superscale, Acceleratooor, Goldilocks]",[1],[1],[Superscale]
541,"[Superscale, Acceleratooor, Goldilocks]",[2],[2],[Acceleratooor]
542,"[Superscale, Acceleratooor, Goldilocks]",[3],[3],[Goldilocks]


In [11]:
voters = pd.read_csv("../Data/Voters_Data.csv")

In [12]:
voters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   voter_address  131 non-null    object
 1   voter_name     90 non-null     object
dtypes: object(2)
memory usage: 2.2+ KB


In [13]:
# Merge voter_name into votes dataframe based on voter_address
votes = votes.merge(voters, on='voter_address', how='left')

In [14]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   vote_id                 1111 non-null   object
 1   voter_address           1111 non-null   object
 2   voting_power            1111 non-null   int64 
 3   vote_timestamp          1111 non-null   object
 4   proposal_id             1111 non-null   object
 5   proposal_title          1111 non-null   object
 6   proposal_type           1111 non-null   object
 7   proposal_choices        1111 non-null   object
 8   proposal_creation_date  1111 non-null   object
 9   proposal_start_date     1111 non-null   object
 10  proposal_end_date       1111 non-null   object
 11  choice                  1111 non-null   object
 12  reason                  182 non-null    object
 13  space                   1111 non-null   object
 14  source                  1111 non-null   object
 15  docu

In [15]:
# Dictionary mapping categories to proposal titles (as given)
category_map = {
    "Eval Algos": [
        "Retro Funding S7 – Dev Tooling – Eval Algos",
        "Retro Funding S7 – Onchain Builders – Eval Algos"
    ],
    "Intent Ratification": [
        "Season 7: Intent Ratification",
        "Ratification of Round 4 Profit Definition"
    ],
    "Mission Approval": [
        "Retro Funding: Dev Tooling Mission",
        "Retro Funding: Onchain Builders Mission"
    ],
    "Upgrade Veto Proposals": [
        "Voting Cycle #17: Veto #1",
        "Voting Cycle #18: Veto #2",
        "Protocol Upgrade #5: Veto #3",
        "Protocol Upgrade #6: Veto #4",
        "Protocol Upgrade #7: Veto #6",
        "Protocol Upgrade #8: Veto #7",
        "Protocol Upgrade #9: Veto #9",
        "Protocol Upgrade #10: Veto #10",
        "Governor Upgrade #1: Improve advanced delegation voting",
        "Governor Update Proposal #2: Improvements to advanced delegation allowance calculations: Veto #8",
        "Governor Update Proposal #3: Enable onchain treasury execution"
    ],
    "Upgrade Proposals": [
        "Upgrade Proposal #11: Holocene Network Upgrade",
        "Protocol Upgrade: Superchain Registry 2.0",
        "Upgrade Proposal #13: OPCM and Incident Response improvements",
        "Upgrade Proposal #14: Isthmus L1 Contracts + MT-Cannon",
        "Upgrade Proposal #15: Isthmus Hard Fork"
    ],
    "Budget Board Ratification": [
        "Season 8 and 9: Budget Board Member Ratification"
    ],
    "Sequencer Revenue": [
        "Allow the Optimism Foundation to Stake a Portion of Sequencer ETH Through Season 8"
    ],
    "Maintenance Upgrade": [
        "Maintenance Upgrade: L1 Pectra Readiness",
        "Maintenance Upgrade: Absolute Prestate Updates for Isthmus Activation & Blob Preimage Fix"
    ],
    "Test Proposals": [
        "Test Vote (Season 6)",
        "Test Veto Vote"
    ],
    "Retro Funding Round 6": [
        "Retro Funding Round 6 - weighing the votes of Guest Voters"
    ]
}

# Optional: Clean your proposal titles for safer matching
votes["proposal_title"] = votes["proposal_title"].str.strip().str.replace("–", "-", regex=False)

# Function to assign category by partial match
def get_category_fuzzy(title):
    title_lower = title.lower()
    for category, titles in category_map.items():
        for ref_title in titles:
            if ref_title.lower() in title_lower:
                return category
    return "Uncategorized"

# Apply to DataFrame
votes["proposal_category"] = votes["proposal_title"].apply(get_category_fuzzy)


In [16]:
# Step 1: Convert to datetime (auto-detects timezone from string)
votes['creation_datetime'] = pd.to_datetime(votes['proposal_creation_date'])

# Step 2: Define timezone-aware datetime ranges (India Standard Time)
ist = pytz.timezone('Asia/Kolkata')

rounds = {
    4: {'start': ist.localize(datetime(2024, 1, 12)), 'end': ist.localize(datetime(2024, 7, 16))},
    5: {'start': ist.localize(datetime(2024, 7, 17)), 'end': ist.localize(datetime(2024, 10, 21))},
    6: {'start': ist.localize(datetime(2024, 10, 22)), 'end': ist.localize(datetime(2025, 7, 31))},
}

# Step 3: Function to assign round_number
def get_round_number(dt):
    for round_num, dates in rounds.items():
        if dates['start'] <= dt <= dates['end']:
            return round_num
    return None  

# Step 4: Apply
votes['round_number'] = votes['creation_datetime'].apply(get_round_number)

# Optional: drop intermediate datetime column
votes.drop(columns=['creation_datetime'], inplace=True)

In [17]:
# Ensure chosen_option_text is lower case for safe comparison
votes['chosen_option_text_lower'] = votes['chosen_option_text'].str.lower()

# is_abstain: true if the user abstained
votes['is_abstain'] = votes['chosen_option_text_lower'].eq('abstain')

# was_veto: true if the user voted "veto"
votes['was_veto'] = votes['chosen_option_text_lower'].eq('veto')

votes.drop(columns=['chosen_option_text_lower'], inplace=True)

In [18]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   vote_id                 1111 non-null   object
 1   voter_address           1111 non-null   object
 2   voting_power            1111 non-null   int64 
 3   vote_timestamp          1111 non-null   object
 4   proposal_id             1111 non-null   object
 5   proposal_title          1111 non-null   object
 6   proposal_type           1111 non-null   object
 7   proposal_choices        1111 non-null   object
 8   proposal_creation_date  1111 non-null   object
 9   proposal_start_date     1111 non-null   object
 10  proposal_end_date       1111 non-null   object
 11  choice                  1111 non-null   object
 12  reason                  182 non-null    object
 13  space                   1111 non-null   object
 14  source                  1111 non-null   object
 15  docu

In [19]:
# Reorder and rename (if needed)
desired_vote_columns = [
    "source",
    "document_role",
    "vote_id",
    "voter_address",
    "voter_name",
    "voting_power",
    "vote_timestamp",
    "proposal_id",
    "proposal_title",
    "proposal_type",
    "proposal_choices",
    "proposal_category",
    "proposal_creation_date",
    "proposal_start_date",
    "proposal_end_date",
    "chosen_option_index",
    "chosen_option_text",
    "round_number",
    "is_abstain",
    "was_veto",
    "reason"
]

# Reorder the columns
votes = votes[desired_vote_columns]


In [20]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   source                  1111 non-null   object
 1   document_role           1111 non-null   object
 2   vote_id                 1111 non-null   object
 3   voter_address           1111 non-null   object
 4   voter_name              879 non-null    object
 5   voting_power            1111 non-null   int64 
 6   vote_timestamp          1111 non-null   object
 7   proposal_id             1111 non-null   object
 8   proposal_title          1111 non-null   object
 9   proposal_type           1111 non-null   object
 10  proposal_choices        1111 non-null   object
 11  proposal_category       1111 non-null   object
 12  proposal_creation_date  1111 non-null   object
 13  proposal_start_date     1111 non-null   object
 14  proposal_end_date       1111 non-null   object
 15  chos

In [21]:
votes.to_json("../Data/Votes_Data.json", orient="records", force_ascii=False, indent=2)

In [22]:
# Filter out rows without a round number (if any)
votes = votes[votes['round_number'].notnull()]

# Save per round
for round_num in sorted(votes['round_number'].unique()):
    round_df = votes[votes['round_number'] == round_num]
    output_path = f"../Data/RPGF{int(round_num)}_Votes_Data.json"
    round_df.to_json(output_path, orient="records", force_ascii=False, indent=2)
    print(f"Saved: {output_path} with {len(round_df)} records.")

Saved: ../Data/RPGF4_Votes_Data.json with 391 records.
Saved: ../Data/RPGF5_Votes_Data.json with 52 records.
Saved: ../Data/RPGF6_Votes_Data.json with 668 records.
